<a href="https://colab.research.google.com/github/Adhihack888/FMML_Project_and_Labs/blob/main/module_01_lab_02_mlpractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

*1)*The accuracy of a machine learning model's validation set can be affected by changes in the percentage of data allocated to the validation set. Let's explore how increasing or reducing the validation set percentage can impact model performance:

1. **Increasing the Validation Set Percentage:**
   - **Pros:** A larger validation set can provide a more reliable estimate of a model's generalization performance. It may help in detecting overfitting, as the model is evaluated on a larger unseen portion of the data.
   - **Cons:** With a larger validation set, you have fewer training examples. This may lead to longer training times and potentially underfitting if the training data is not representative enough. Also, it may not be practical if you have limited data.

   In summary, increasing the validation set percentage tends to make the validation accuracy a more trustworthy indicator of model performance, but it may come at the cost of potentially slower training and reduced model capacity.

2. **Reducing the Validation Set Percentage:**
   - **Pros:** More data for training can lead to better model performance, especially when you have a limited dataset. Faster training times are also a benefit.
   - **Cons:** A smaller validation set might not provide a reliable estimate of how well the model will generalize to unseen data. It can make the validation accuracy more susceptible to randomness and might not effectively detect overfitting.

   Reducing the validation set percentage often results in a validation accuracy that is more variable and potentially less indicative of the model's true performance on new data.

The choice of the validation set percentage depends on various factors, including the size of your dataset, the complexity of your model, and the computational resources available. Typically, a common split is 70-80% for training and 20-30% for validation, but this can vary.

In practice, you might also use techniques like cross-validation or stratified sampling to mitigate some of the variability in validation accuracy when working with smaller datasets or facing other challenges. The key is to strike a balance that allows your model to train effectively while still providing a good estimate of its generalization performance.

*2)*The size of the train and validation sets can significantly affect how well we can predict the accuracy on the test set using the validation set. Here's how it works:

1. **Larger Train Set:**
   - When you have a larger training set, the model has more data to learn from. This often results in better model performance because the model can capture more patterns and nuances in the data.
   - A larger training set can also help reduce overfitting because the model has more diverse examples to generalize from.
   - However, if the validation set remains relatively small, it might not provide a reliable estimate of how well the model will perform on the much larger test set. In this case, the validation accuracy might not be a precise predictor of test set accuracy.

2. **Larger Validation Set:**
   - A larger validation set can provide a more reliable estimate of how well the model generalizes to unseen data. It reduces the variability in the estimated validation accuracy.
   - However, if the training set is relatively small, a large validation set can lead to a smaller training set, potentially causing the model to underfit due to a lack of training data.

3. **Balanced Split:**
   - Ideally, you want a balanced split between the training and validation sets, taking into account the size of your dataset. A common split is 70-80% for training and 20-30% for validation.
   - A balanced split allows the model to learn from a substantial portion of the data while still providing a reasonably large validation set for estimating performance.

In summary, the size of the train and validation sets should be chosen carefully to strike a balance between training the model effectively and accurately estimating its generalization performance. The key is to ensure that both sets are representative of the overall data distribution, and the choice often depends on factors like the size of your dataset, the complexity of your model, and the specific problem you are addressing. Additionally, techniques like cross-validation can help provide a more robust estimate of model performance, especially when data is limited.

*3)*There isn't a one-size-fits-all answer to what percentage should be reserved for the validation set because the optimal split between the training and validation sets depends on various factors, including the size of your dataset, the complexity of your model, and your specific goals. However, a common and reasonable starting point is to reserve around 20-30% of your data for the validation set.

Here are some considerations to help you decide:

1. **Dataset Size:** If you have a large dataset (thousands or more samples), you can afford to allocate a smaller percentage to the validation set, as you'll still have a substantial amount of data for training. In this case, 20% for validation might be sufficient.

2. **Model Complexity:** If you're working with a very complex model (deep neural networks, for example), you may want to allocate a larger validation set to ensure better estimation of model generalization. In such cases, 30% or even more for validation can be beneficial.

3. **Limited Data:** If you have a relatively small dataset, you might consider a larger validation set percentage to obtain a more reliable estimate of model performance. You could go as high as 30% or even 40% for validation.

4. **Cross-Validation:** Another approach, especially when data is limited, is to use techniques like k-fold cross-validation. This involves splitting your data into k subsets and using each subset as the validation set while training on the remaining data. This way, you can make the most out of your data and obtain more robust performance estimates.

In practice, it's often helpful to experiment with different splits and techniques to find the balance that works best for your specific problem. The key is to ensure that your validation set is representative of your data and provides a good estimate of how well your model generalizes to unseen examples.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


*1)*Yes, averaging the validation accuracy across multiple splits, such as in k-fold cross-validation, can indeed provide more consistent and reliable results when evaluating a machine learning model. Here's why it can be beneficial:

1. **Reducing Variance:** Averaging validation accuracy over multiple splits helps reduce the impact of randomness and variability in the data. In a single train-validation split, the specific examples chosen for the validation set can have a significant influence on the observed accuracy. By averaging over multiple splits, you get a more stable estimate of the model's performance.

2. **Better Generalization Estimate:** Cross-validation allows the model to be trained and validated on different subsets of the data, providing a more comprehensive assessment of how well the model generalizes to unseen data. This is especially valuable when you have a limited dataset, as it maximizes the use of available data.

3. **Robustness to Data Distribution:** It helps ensure that your model's performance isn't overly dependent on the peculiarities of a single data split. Different splits provide a more holistic view of how well your model is likely to perform on various subsets of your data.

4. **Model Selection:** When comparing multiple models or tuning hyperparameters, cross-validation can aid in selecting the best-performing model or settings more reliably.

However, it's important to note that cross-validation can be computationally expensive, especially for large datasets or complex models. You'll need to balance the desire for more reliable estimates with the available computational resources.

In summary, averaging validation accuracy across multiple splits, such as in k-fold cross-validation, is a common and effective technique to obtain more consistent and robust performance estimates for your machine learning models, making it a valuable tool for model assessment and selection.

*2)*Averaging validation accuracy across multiple splits, as done in techniques like k-fold cross-validation, provides a more accurate estimate of the model's generalization performance compared to a single train-validation split. However, it's essential to clarify the terminology here:

- **Validation Accuracy:** This is an estimate of how well your model performs on data it hasn't seen during training but is drawn from the same dataset. It's used to tune hyperparameters, assess model performance, and make model selection decisions during the development phase.

- **Test Accuracy:** This is a measure of how well your model performs on entirely unseen, independent data that it has never encountered during training or validation. The test set is used to evaluate the final, unbiased performance of your model.

Averaging validation accuracy across multiple splits, while helpful, is still an estimate of how well your model might generalize to unseen data. It provides a more robust and reliable estimate than a single split but is still not a direct measure of test accuracy.

To get an accurate estimate of test accuracy, you need to reserve a separate, independent test dataset that is not used during model development or hyperparameter tuning. This test dataset should represent the real-world data your model will encounter when deployed. Testing on this dataset gives you a true measure of how well your model performs in practice.

In summary, cross-validation and averaging validation accuracy across multiple splits improve your confidence in your model's generalization performance but are not a replacement for a dedicated test set to measure actual test accuracy. The test set remains essential for assessing how well your model performs in real-world scenarios.

*3)*The number of iterations, also known as "folds," in techniques like k-fold cross-validation can have an impact on the accuracy of the estimate of your model's performance. However, it's not necessarily the case that a higher number of iterations always leads to a better estimate. Here's how it works:

1. **Effect of Number of Iterations:**
   - **More Iterations:** Increasing the number of iterations (folds) in cross-validation generally provides a more accurate and robust estimate of your model's performance. This is because the data is split into more subsets, and your model gets to train and validate on a larger variety of data combinations.
   - **Fewer Iterations:** Using fewer iterations can be computationally faster but may result in a less reliable estimate. With fewer iterations, the validation sets in each split are larger, which can lead to more variability in the estimate.

2. **Diminishing Returns:** While increasing the number of iterations generally improves the estimate, there are diminishing returns. At some point, adding more folds may not significantly enhance the estimate but will increase computational cost.

3. **Trade-Off:** You need to strike a balance between the number of iterations and computational resources available. Very high values of k (e.g., leave-one-out cross-validation with k equal to the number of data points) can be computationally expensive and may not offer a substantial improvement over more moderate values of k.

In practice, common values for k in k-fold cross-validation are 5, 10, or even 3, depending on the dataset size and complexity. These values strike a balance between obtaining a reliable estimate of model performance and not overly burdening computational resources.

So, while higher iterations generally lead to a better estimate, there's a trade-off to consider. You should choose a reasonable number of iterations that balances computational cost with the need for a reliable performance estimate for your specific modeling task.

*4)*Increasing the number of iterations in techniques like k-fold cross-validation can help mitigate the impact of having a very small training dataset or validation dataset to some extent, but it has its limitations. Here's how it works:

**Small Training Dataset:**
- If you have a very small training dataset, increasing the number of iterations can help in the sense that each fold will still use a larger portion of your data for training and validation.
- More iterations mean that your model gets to see different subsets of the data during each fold, which can provide a more comprehensive view of the data.

**Small Validation Dataset:**
- When your validation dataset is small, increasing iterations can also be beneficial. With more folds, each validation set will have a larger sample size, reducing the potential for high variability in performance estimates.
- This can make the validation accuracy more reliable and representative of your model's true performance.

However, there are limitations to consider:

1. **Computational Cost:** Increasing the number of iterations significantly increases the computational cost. If you have extremely limited computational resources, you might not be able to perform a very high number of iterations.

2. **Data Scarcity:** If both your training and validation datasets are extremely small, increasing iterations might still not provide enough data for your model to learn meaningful patterns. In such cases, there's a risk of the model underfitting due to a lack of training data.

3. **Model Complexity:** The impact of increasing iterations can also depend on your model's complexity. Highly complex models may require larger training datasets to learn effectively, and increasing iterations alone may not compensate for this limitation.

In conclusion, while increasing the number of iterations can help when dealing with small training or validation datasets, it's not a complete solution, especially if the data is extremely limited. The choice of the number of iterations should be made based on a balance between computational resources and the need for reliable performance estimates. Additionally, in cases of severe data scarcity, it's essential to consider data augmentation techniques or alternative strategies to address the limitations of a small dataset.